# Adding Dynamic Features To Improve Energy Usage Predictions (v2)

## Part 1: Load and examine the data

In [ ]:
data_bucket = 'doughudgeon-mlforbusiness' # change the name odf your bucket
subfolder = 'ch07'
s3_data_path = f"s3://{data_bucket}/{subfolder}/data"
s3_output_path = f"s3://{data_bucket}/{subfolder}/output"

In [ ]:
%matplotlib inline

import sys
from dateutil.parser import parse
import json
import random
import datetime
import os

import pandas as pd                               
import boto3
import s3fs
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# correspond to Version 2.x of the SageMaker Python SDK
# Check the latest version of SageMaker
if int(sagemaker.__version__.split('.')[0]) == 2:
    print("Version is good")
else:
    !{sys.executable} -m pip install --upgrade sagemaker
    print("Installing latest SageMaker Version. Please restart the kernel")
    

role = sagemaker.get_execution_role()
s3 = s3fs.S3FileSystem(anon=False)
s3_data_path = f"s3://{data_bucket}/{subfolder}/data"
s3_output_path = f"s3://{data_bucket}/{subfolder}/output"

In this chapter we will be dealing with 5 files. Our meter readings have already been summarised into daily totals and can be found in "Meter Data.csv". We also have a "Site Categories.csv" file which records whether each site belongs to the Retail, Industrial, or Transport industries. We will use this as a static "Category" feature. There is a further file which contains time series data regarding holidays. This is "Site Holidays.csv". Finally we have maximum temperatures in "Site Maxima.csv" These will be our models "dynamic features".

In [ ]:
# First we check our meter data
daily_df = pd.read_csv(f's3://{data_bucket}/{subfolder}/meter_data_daily.csv', index_col=0, parse_dates=[0])
daily_df.index.name = None
daily_df.head()

In [ ]:
print(daily_df.shape)
print(f'Time series starts at {daily_df.index[0]} and ends at {daily_df.index[-1]}')

In [ ]:
category_df = pd.read_csv(f's3://{data_bucket}/{subfolder}/site_categories.csv',index_col=0).reset_index(drop=True)
print(category_df.shape)
print(category_df.Category.unique())
category_df.head()

In [ ]:
holiday_df = pd.read_csv(f's3://{data_bucket}/{subfolder}/site_holidays.csv', index_col=0, parse_dates=[0])
print(holiday_df.shape)
print(f'Time series starts at {holiday_df.index[0]} and ends at {holiday_df.index[-1]}')
holiday_df.loc['2018-12-22':'2018-12-27']

In [ ]:
max_df = pd.read_csv(f's3://{data_bucket}/{subfolder}/site_maximums.csv', index_col=0, parse_dates=[0])
print(max_df.shape)
print(f'Time series starts at {max_df.index[0]} and ends at {max_df.index[-1]}')
max_df.loc['2018-12-22':'2018-12-27']

## Part 2: Get the data in the right shape

In [ ]:
# We need to do this to set freq='D' on the index:
daily_df = daily_df.resample('D').sum()
daily_df = daily_df.replace([0],[None])

In [ ]:
daily_df[daily_df.isnull().any(axis=1)].index

Some sites have missing values for November 2017. We will not impute these but instead train with missing values.
During the prediction step we will only use values from December. So daily_df is already in the right shape!

In [ ]:
# How about categoricals?
print(f'{len(category_df[category_df.isnull().any(axis=1)])} sites with missing categories.')

In [ ]:
# Dynamic features?
print(f'{len(holiday_df[holiday_df.isnull().any(axis=1)])} days with missing holidays.')
print(f'{len(max_df[max_df.isnull().any(axis=1)])} days with missing maximum temperatures.')

In [ ]:
# So we have to impute missing temperatures. Weather does not follow a weekly cycle like energu usage does,
# but pandas has a very nice way to impute missing values for this very situation:
max_df = max_df.interpolate(method='time')
print(f'{len(max_df[max_df.isnull().any(axis=1)])} days with missing maximum temperatures. Problem solved!')

In [ ]:
# Confirm visually we are dealing with the same or similar data to chapter 6:
print('Number of time series:',daily_df.shape[1])
fig, axs = plt.subplots(6, 2, figsize=(20, 20), sharex=True)
axx = axs.ravel()
indices = [0,1,2,3,26,27,33,39,42,43,46,47]
for i in indices:
    plot_num = indices.index(i)
    daily_df[daily_df.columns[i]].loc["2017-11-01":"2019-02-28"].plot(ax=axx[plot_num])
    axx[plot_num].set_xlabel("date")    
    axx[plot_num].set_ylabel("kW consumption")

## Part 3: Create Train and Test Datasets

In [ ]:
cats = list(category_df.Category.astype('category').cat.codes)
print(cats)

In [ ]:
# stations = list(station_df['Station Number'].astype('category').cat.codes)
# assert len(stations)==48
# print(stations)

In [ ]:
usage_per_site = [daily_df[col] for col in daily_df.columns]

print(f'Time series covers {len(usage_per_site[0])} days.')
print(f'Time series starts at {usage_per_site[0].index[0]}')
print(f'Time series ends at {usage_per_site[0].index[-1]}') 
usage_per_site[0][:10]

In [ ]:
# Do the same for our dynamic features
hols_per_site = [holiday_df[col] for col in holiday_df.columns]
    
print(f'Time series covers {len(hols_per_site[0])} days.')
print(f'Time series starts at {hols_per_site[0].index[0]}')
print(f'Time series ends at {hols_per_site[0].index[-1]}') 
hols_per_site[0][:10]

In [ ]:
max_per_site = [max_df[col] for col in max_df.columns]
    
print(f'Time series covers {len(max_per_site[0])} days.')
print(f'Time series starts at {max_per_site[0].index[0]}')
print(f'Time series ends at {max_per_site[0].index[-1]}') 
max_per_site[0][:10]

In [ ]:
freq = 'D'
prediction_length = 28

from datetime import timedelta

start_date = pd.Timestamp("2017-11-01", freq=freq)
end_training = pd.Timestamp("2019-01-31", freq=freq)
end_testing = end_training + timedelta(days=prediction_length)

print(f'End training: {end_training}, End testing: {end_testing}')

In [ ]:
def write_dicts_to_s3(path, data):
    with s3.open(path, 'wb') as f:
        for d in data:
            f.write(json.dumps(d).encode("utf-8"))
            f.write("\n".encode('utf-8'))

In [ ]:
# NOTE: We have missing values in ts for November only.
#       Dynamic features must have numeric values for every entry.
training_data = [
    {
        "cat": [cat],
        "start": str(start_date),
        "target": ts[start_date:end_training].tolist(),
        "dynamic_feat": [
            hols[start_date:end_training].tolist(),
            maxes[start_date:end_training].tolist(),
        ] # Note: List of lists
    }
    for cat,ts,hols,maxes in zip(cats, usage_per_site, hols_per_site, max_per_site)
]

test_data = [
    {
        "cat": [cat],
        "start": str(start_date),
        "target": ts[start_date:end_testing].tolist(),
        "dynamic_feat": [
            hols[start_date:end_testing].tolist(),
            maxes[start_date:end_testing].tolist(),
        ] # Note: List of lists
    }
    for cat,ts,hols,maxes in zip(cats, usage_per_site, hols_per_site, max_per_site)
]
            
write_dicts_to_s3(f'{s3_data_path}/train/train.json', training_data)
write_dicts_to_s3(f'{s3_data_path}/test/test.json', test_data)

## Part 4: Set up session and configure model

In [ ]:
s3_output_path = f's3://{data_bucket}/{subfolder}/output'
sess = sagemaker.Session()
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", sess.boto_region_name, "latest")

data_channels = {
    "train": f"{s3_data_path}/train/",
    "test": f"{s3_data_path}/test/"
}
np.random.seed(42)
random.seed(42)

In [ ]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
from sagemaker.serializers import IdentitySerializer

In [ ]:
# Reuse the class from Chapter 6:
class DeepARPredictor(sagemaker.predictor.Predictor):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, 
                         #serializer=JSONSerializer(),
                         serializer=IdentitySerializer(content_type="application/json"),
                         **kwargs)
        
    def predict(self, ts, cat=None, dynamic_feat=None, 
                num_samples=100, return_samples=False, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + ts.index.freq
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)
    
    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None)
        
        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')
    
    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode('utf-8'))['predictions'][0]
        prediction_length = len(next(iter(predictions['quantiles'].values())))
        prediction_index = pd.date_range(start=prediction_time, freq=freq, periods=prediction_length)
        if return_samples:
            dict_of_samples = {'sample_' + str(i): s for i, s in enumerate(predictions['samples'])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(data={**predictions['quantiles'], **dict_of_samples}, index=prediction_index)

    def set_frequency(self, freq):
        self.freq = freq
        
def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]        

def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat        
    return obj

## Part 5a: Create model without using additional datasets

First, we will establish a baseline without categorical or dynamic features. Note that this cell is commented out as you only need to run it if you want to see the MAPE without incorporating additional datasets.

In [ ]:
# estimator = sagemaker.estimator.Estimator(
#     sagemaker_session=sess,
#     image_name=image_name,
#     role=role,
#     train_instance_count=1,
#     train_instance_type='ml.c5.2xlarge', # $0.476 per hour as of Jan 2019.
#     base_job_name='ch7-energy-usage-baseline',
#     output_path=s3_output_path
# )

# estimator.set_hyperparameters(
#     cardinality='ignore', # DISABLES CATEGORICALS FOR BASELINE
#     context_length="90",
#     prediction_length=str(prediction_length),
#     time_freq=freq,
#     epochs="400",
#     early_stopping_patience="40",
#     mini_batch_size="64",
#     learning_rate="5E-4",
#     num_dynamic_feat="ignore", # DISABLE DYNAMIC FEATURES FOR BASELINE
# )

# estimator.fit(inputs=data_channels, wait=True)

# endpoint_name = 'energy-usage-baseline'

# try:
#     sess.delete_endpoint(
#         sagemaker.predictor.RealTimePredictor(endpoint=endpoint_name).endpoint)
#     print('Warning: Existing endpoint and configuration deleted to make way for your new endpoint.')
#     from time import sleep
#     sleep(30)
# except:
#     pass

# predictor = estimator.deploy(
#     initial_instance_count=1,
#     instance_type='ml.m5.large',
#     predictor_cls=DeepARPredictor,
#     endpoint_name=endpoint_name)

# # Gather 28 day predictions for all timeseries
# usages = [ts[end_training+1:end_training+28].sum() for ts in usage_per_site]

# predictions= []
# for s in range(len(usage_per_site)):
#     # call the end point to get the 28 day prediction
#     predictions.append(
#         predictor.predict(
#             ts=usage_per_site[s][start_date+30:end_training],
#         )['0.5'].sum()
#     )
    
# print(f'MAPE: {round(mape(usages, predictions),1)}%')    

## Part 5b: Model incorporating additional datasets

In [ ]:
%%time
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sess,
    image_uri=image_name,
    role=role,
    instance_count=1,
    instance_type='ml.c5.2xlarge', # $0.476 per hour as of Jan 2019.
    base_job_name='ch7-energy-usage-dynamic',
    output_path=s3_output_path
)

estimator.set_hyperparameters(
    context_length="90",
    prediction_length=str(prediction_length),
    time_freq=freq,
    epochs="400",
    early_stopping_patience="40",
    mini_batch_size="64",
    learning_rate="5E-4",
    num_dynamic_feat=2,
)

estimator.fit(inputs=data_channels, wait=True)

## Part 6. Making predictions from the model that incorporates additional datasets

In [ ]:
endpoint_name = 'energy-usage-dynamic'

try:
    sess.delete_endpoint(
        sagemaker.predictor.Predictor(endpoint=endpoint_name).endpoint)
    print('Warning: Existing endpoint and configuration deleted to make way for your new endpoint.')
    from time import sleep
    sleep(30)
except:
    pass

In [ ]:
%%time
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    predictor_cls=DeepARPredictor,
    endpoint_name=endpoint_name)

In [ ]:
# Test prediction: (Delete endpoint configuration if retrying)
predictor.predict(
    cat=[cats[0]],
    ts=usage_per_site[0][start_date+datetime.timedelta(30):end_training],
     dynamic_feat=[
             hols_per_site[0][start_date+datetime.timedelta(30):end_training+datetime.timedelta(28)].tolist(),
             max_per_site[0][start_date+datetime.timedelta(30):end_training+datetime.timedelta(28)].tolist(),
         ],
    quantiles=[0.1, 0.5, 0.9]
).head()

In [ ]:
# Gather 28 day predictions for all timeseries
usages = [ts[end_training+datetime.timedelta(1):end_training+datetime.timedelta(28)].sum() for ts in usage_per_site]

predictions= []
for s in range(len(usage_per_site)):
    # call the end point to get the 28 day prediction
    predictions.append(
        predictor.predict(
            cat=[cats[s]],
            ts=usage_per_site[s][start_date+datetime.timedelta(30):end_training],
            dynamic_feat=[
                hols_per_site[s][start_date+datetime.timedelta(30):end_training+datetime.timedelta(28)].tolist(),
                max_per_site[s][start_date+datetime.timedelta(30):end_training+datetime.timedelta(28)].tolist(),
             ]
        )['0.5'].sum()
    )

for p,u in zip(predictions,usages):
    print(f'Predicted {p} kwh but usage was {u} kwh.')

In [ ]:
print(f'MAPE: {round(mape(usages, predictions),1)}%')

That's a really impressive improvement from 21%. What does this look like visually?

In [ ]:
def plot(
    predictor, 
    site_id,
    end_training=end_training, 
    plot_weeks=12,
    confidence=80
):
    low_quantile = 0.5 - confidence * 0.005
    up_quantile = confidence * 0.005 + 0.5
    target_ts = usage_per_site[site_id][start_date+datetime.timedelta(30):]
    dynamic_feats = [
            hols_per_site[site_id][start_date+datetime.timedelta(30):].tolist(),
            max_per_site[site_id][start_date+datetime.timedelta(30):].tolist(),
        ]
        
    plot_history = plot_weeks * 7

    fig = plt.figure(figsize=(20, 3))
    ax = plt.subplot(1,1,1)
    
    prediction = predictor.predict(
        cat = [cats[site_id]],
        ts=target_ts[:end_training],
        dynamic_feat=dynamic_feats,
        quantiles=[low_quantile, 0.5, up_quantile])
                
    target_section = target_ts[end_training-datetime.timedelta(plot_history):end_training+datetime.timedelta(prediction_length)]
    target_section.plot(color="black", label='target')
    
    ax.fill_between(
        prediction[str(low_quantile)].index, 
        prediction[str(low_quantile)].values, 
        prediction[str(up_quantile)].values, 
        color="b", alpha=0.3, label='{}% confidence interval'.format(confidence)
    )  
    
    ax.set_ylim(target_section.min() * 0.5, target_section.max() * 1.5)

In [ ]:
# Plot 4 of each category:
indices = [2,26,33,39,42,47,3]
for i in indices:
    plot_num = indices.index(i)
    plot(
        predictor,
        site_id=i,
        plot_weeks=6,
        confidence=80
    )

## Part 7. Remove the Endpoints

In [ ]:
# Remove the Endpoint
# Comment out this cell to remove the endpoint if you want the endpoint to exist after "run all"

#sess.delete_endpoint('energy-usage-baseline')
sess.delete_endpoint('energy-usage-dynamic')